In [71]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import display, HTML
import math
import itertools

# Import data

### Weather:
- weather data from within the observed avalanche zone (back to Oct. 2010)
Source: [MesoWest](http://mesowest.utah.edu/) Mill D North station (MLDU1)

### Observed avalanches:
- Avalanches obtained from [Utah Avalanche Center](https://utahavalanchecenter.org/avalanche-list) SLC area avalanches
- Avalanches merged with [Wasatch Backcountry Skiing map](http://wbskiing.com/) to obtain lat/lon. Merge was performed using a fuzzy match on the ski run name
- Lat/Lon was used to obtain elevation, aspect, slope using [NED DEM](https://gis.utah.gov/data/elevation-terrain-data/) file and GRASS GIS

### Avalanche advisories:
- Advisories released daily provide an avalanche risk score from 1-5 (green to black)
- Data obtained by scraping [advisory rose](https://utahavalanchecenter.org/advisory/salt-lake/02/13/2017) images from UAC archive
- Each image was then processed to obtain a pixel sample within each elevation/aspect permutation and stored with the date. The pixel values were translated to the avalanche risk score.

### Ski Runs:
- Runs obtained from [Wasatch Backcountry Skiing map](http://wbskiing.com/)
- Lat/Lon dropped as to not publish this data. I had to dig through obfuscated javascript to get it.
- Slope, aspect, and elevation will be used

In [135]:
#weather
weather = pd.read_csv('full_weather.csv')

#observed avalanches
avalanches = pd.read_csv('final_avalanches.csv')

#avalanche advisories
adv = pd.read_csv('advisories.csv')

#ski run info
runs = pd.read_csv('run_info.csv')


In [136]:
#convert elevations from meters to feet
def to_feet(m):
    return round(m * 3.28084)

feet = np.vectorize(to_feet)

runs['avy_elevat'] = feet(runs['avy_elevat'])


In [137]:
def assign_position(d):
    return "-".join(["rose", str(get_el(d['avy_elevat'])), str(get_aspect(d['aspect']))])
    
def get_aspect(asp):
    return int(math.floor((asp / 360.0) * 8))

def get_el(el):
    if(el < 8000):
        return 0
    elif(el >= 8000 and el < 9500):
        return 1
    elif(el >= 9500):
        return 2

#print assign_position(adv[0])
runs['rose'] =  runs.apply(assign_position, axis=1)
    

In [138]:
# permutations of aspect/elevation
elevations = map(lambda x: str(x), range(0,2))
aspects = map(lambda x: str(x), range(0,8))
perms = map('-'.join, itertools.product(elevations, aspects))
perms = map(lambda x: 'rose-' + x, perms)

def get_random_obs(p):
    samp = pd.DataFrame()
    samp['date'] = adv[adv[p] == 0]['date'].sample(100)
    samp['rose'] = p
    return samp
    
samples = map(get_random_obs, perms)


In [141]:

def get_run_sample(row):
    pd.concat([row, runs[runs['rose'] == row['rose']].sample(1)], axis=1)
    #print runs[runs['rose'] == row['rose']].sample(1)

m_samples = pd.concat(samples)
runsamp = []
for index, row in m_samples.iterrows():
    #get_run_sample(row)
    #runsamp.append()
    pass

    
#print(runsamp)
display(runs)
#get_run_sample('rose-0-0')

,slope,aspect,avy_elevat,rose
0,34.22575,45.54170,8517.0,rose-1-1
1,31.88675,169.58725,8991.0,rose-1-3
2,32.92081,114.66229,9128.0,rose-1-2
3,44.27121,19.23148,8476.0,rose-1-0
4,36.45871,353.02026,8476.0,rose-1-7
5,22.16785,50.06602,8051.0,rose-1-1
6,26.44122,299.13763,10289.0,rose-2-6
7,44.49792,97.74413,10203.0,rose-2-2
8,20.88771,251.08237,10340.0,rose-2-5
9,40.89746,84.76679,10014.0,rose-2-1
